In [1]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout, exit, stderr
import numpy as np
from tqdm import tqdm

from tqdm import tqdm_notebook

you got numpy


In [2]:
input_name = 'st_kras_0'
output_name = 'st_kras_1'

# Load up restart files

In [3]:

pdb = PDBFile('./output.pdb')
complex_state = XmlSerializer.deserialize(open(input_name+'_state.xml').read())
complex_system = XmlSerializer.deserialize(open(input_name+'_system.xml').read())



In [4]:

integrator = LangevinIntegrator(310*kelvin, 1/picosecond, 0.002*picoseconds)

platform = Platform.getPlatformByName('CUDA')
prop = {'CudaPrecision':'single', 'CudaDeviceIndex':'0'}
simulation = Simulation(pdb.topology, complex_system, integrator, platform, prop)

Reporter and load the state object (i.e. restart)

In [5]:
simulation.reporters.append(DCDReporter(output_name+'_traj.dcd', 125000))
simulation.reporters.append(StateDataReporter(output_name+'.log', 10000,
                                              step=True,
                                              time=True,
                                              potentialEnergy=True,
                                              temperature=True,
                                              totalSteps=5000000,
                                              speed=True))


print('loadstate')
simulation.loadState(input_name+'_state.xml')


loadstate


In [6]:
weights = [float(i) for i in open(input_name+'_temp.dat').readlines()[-1].strip('\n').split('\t')[2:]]

In [7]:
st = SimulatedTempering(simulation,
                            numTemperatures=30,
                            tempChangeInterval=5,
                            minTemperature=310*kelvin,
                            maxTemperature=360*kelvin,
                            reportInterval=500,
                            reportFile=(str(output_name)+'_temp.dat'))
st._weights = weights
st._updateWeights = False

0.0001
lewis


In [8]:
num_nano = 150
ns = 500000

for _ in tqdm_notebook(range(num_nano)):
    st.step(ns)

KeyboardInterrupt: 

In [ ]:
with open(output_name+'_system.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                complex_system
            )
    )

simulation.saveState(output_name+'_state.xml')
